这个文件是我训练用的主文件，也就是将数据预处理过的文件读入进来进行训练。该文件主要由三个部分组成：第一部分，神经网络的搭建；第二部分，一次训练的操作；第三部分，主函数，进行多折训练。
对于本次任务，我将其当作回归任务来进行，通过神经网络的学习，传入一部电影的特性，返回一个0到5的浮点数，并将浮点数进行处理与电影的真实评分相比较来评估模型的可行性。
首先下面这些是我用到的所有库，包括基本的numpy、pandas、matplotlib、torch，以及sklearn中的KFold函数用于生成训练集和测试集和学习率退火器stepLR用于在训练过程中调整学习率。

In [43]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import os

下面是我定义的超参数和两个辅助性函数。其中to_near_half的功能是将浮点数近似到具体它最近的0.5的倍数，accuracy的功能是评估模型的准确性，传入的是两个张量，分别为真实评分以及预测评分，假如预测评分落在真实评分不超过0.5的范围内，则可以认定该判定是准确的。

In [44]:
epoch = 300
Lr = 0.003
fig = plt.figure()
def to_near_half(a):
    return torch.round(a * 2) / 2
def accuracy(a, b, threshold = 0.5):
    return sum(abs(a - b) <= threshold)/ len(a)

<Figure size 640x480 with 0 Axes>

下面是我自定义的一个网络MyNet。这个网络有3个比较特别的参数：alpha,beta,gamma。他们分别是：用户的评分偏好数组，电影评分偏好数组，标签评分偏好数组。由于userid并不是一个具有连续意义的分类属性，因此将该属性直接传入全连接层进行学习，并不能体现出不同的userid所带来的影响。因此，这里我设置了一个alpha离散函数，对于每一个userid，作为索引传入数组可以得到一个对应的值。这个值是可以被学习的，并且该值的学习只与每一个userid本身有关，这就达到了将分类的、没有连续意义的属性userid转化为一个具有连续意义、可以被全连接神经网络学习的变量——偏好。对于电影的评分偏好以及标签的评分偏好也是如此。
此外，在前向传播的过程中，我利用了dropout策略和残差网络的策略进行学习，以期得到更好的网络表达能力以及学习效果，激活函数使用gelu，以避免全连接神经网络的退化。


In [45]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.alpha = torch.nn.Parameter(torch.zeros(1000))
        self.beta = torch.nn.Parameter(torch.zeros(10000))
        self.gamma = torch.nn.Parameter(torch.zeros(24))
        self.g = torch.nn.Parameter(torch.tensor(1.0))
        self.f1 = torch.nn.Linear(73,64)
        self.f2 = torch.nn.Linear(64,32)
        self.res = torch.nn.Linear(32,32)
        self.f6 = torch.nn.Linear(32,1)
        self.dropout = torch.nn.Dropout(p=0.05,inplace = False)
        self.relu = torch.nn.ReLU()
        self.gelu = torch.nn.GELU()
        
        
    def forward(self, x):
        # return torch.ones_like(x, dtype = float) * 3.5
        x = x.clone()
        a = x[:,0].int()
        b = x[:,1].int()
        x[:,0] = self.alpha[a]
        x[:,1] = self.beta[b]
        for i in range(3, 23):
            x[:, i] *= self.gamma[i]
        x = self.gelu(self.f1(x))
        x = self.gelu(self.f2(x))
        x = self.dropout(x)#dropout策略
        x = self.gelu(self.res(x) + x)#残差网络
        return self.f6(x) + self.g

下面是我每一折测试需要调用的函数one_test。传入的参数是折数which，训练集属性x_train，测试集属性x_test，训练集标签y_train,测试集标签y_test。首先实例化网络Q，定义Adam优化器，定义损失函数，然后就进行epoch次迭代。在epoch次迭代的过程中，从第100个epoch之后，每一个epoch都进行一次测试集和训练集的损失测量，并在epoch完成后将图像画出并保存，最后计算准确率并输出。（从100开始记录的原因是前面的损失下降过于迅速，以至于将后续的损失下降给淡化了，因此从较后面开始记录图像）

In [46]:
def one_test(which, x_train, y_train, x_test, y_test):
    Q = MyNet()
    optimizer = torch.optim.Adam(Q.parameters(), lr = Lr, weight_decay=0.006)
    scheduler = StepLR(optimizer=optimizer, step_size= 25, gamma = 0.95, last_epoch = -1)
    loss_f = F.mse_loss
    loss_table = []
    loss_table1 = []
    for _ in range(epoch):
        y_predict = Q(x_train)[:,0]
        loss = loss_f(y_predict, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        if _ > 200:
            loss_table.append(float(torch.mean(loss_f(Q(x_train)[:,0], y_train))))
            loss_table1.append(float(torch.mean(loss_f(Q(x_test)[:,0], y_test))))
    if which % 4 == 0:
        ax = fig.add_subplot(2, 2, 4)
    else:
        ax = fig.add_subplot(2, 2, which % 4)
    ax.plot(loss_table)
    ax.plot(loss_table1)
    if which <= 4:
        fig.savefig("fig.png")
        if which == 4:
            fig.clear()
    if which > 4 and which <= 8:
        fig.savefig("fig1.png")
        if which == 8:
            fig.clear()
    if which > 8:
        fig.savefig("fig2.png")
    test_loss = loss_f(Q(x_train)[:,0], y_train)
    test_loss1 = loss_f(Q(x_test)[:,0], y_test)
    acu = accuracy(to_near_half(Q(x_test)[:, 0]), y_test)
    print("第%d折训练集均方损失为:%f,测试集均方损失为:%f,准确率为:%f"%(which, test_loss, test_loss1, acu))

以下便是程序的主函数，首先定义分折器kf，读取处理好的表格cb_table2.csv用于后续的处理。

In [47]:
kf=KFold(n_splits=10, shuffle = True)
which=0#折数
df = torch.tensor(np.array(pd.read_csv("cb_table2.csv"))[:,1:]).float()

下面是利用kf分出训练集和测试集，并进行训练的过程。训练的结果见下。

In [48]:
for train,test in kf.split(df):#对于十折中每一套数据都进行一次测试，最终取平均值
    x_train = df[train,:-1]
    y_train = df[train,-1]
    x_test = df[test,:-1]
    y_test = df[test,-1]
    which+=1
    one_test(which, x_train, y_train, x_test, y_test)

第1折训练集均方损失为:0.667102,测试集均方损失为:0.745438,准确率为:0.654601
第2折训练集均方损失为:0.669103,测试集均方损失为:0.762557,准确率为:0.647858
第3折训练集均方损失为:0.675172,测试集均方损失为:0.763368,准确率为:0.649246
第4折训练集均方损失为:0.673670,测试集均方损失为:0.737143,准确率为:0.650635
第5折训练集均方损失为:0.673030,测试集均方损失为:0.743662,准确率为:0.653114
第6折训练集均方损失为:0.683083,测试集均方损失为:0.740058,准确率为:0.653610
第7折训练集均方损失为:0.682630,测试集均方损失为:0.743050,准确率为:0.648021
第8折训练集均方损失为:0.676968,测试集均方损失为:0.734518,准确率为:0.652385
第9折训练集均方损失为:0.678084,测试集均方损失为:0.737599,准确率为:0.655856
第10折训练集均方损失为:0.677078,测试集均方损失为:0.739939,准确率为:0.650005
